In [147]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os, string
import csv
import pymongo
import time
import json 

# Extract (Scrape) Data

#### *Does not need to be loaded to pymongo* - can be scraped on the spot for the Project 2

In [2]:
# currentDirectory = os.path.dirname(__file__)
# executable_path = {"executable_path": f"{currentDirectory}\chromedriver.exe"}
executable_path = {'executable_path': 'chromedriver.exe'} #notebook version
browser = Browser('chrome', **executable_path, headless=True)

### Scrape Schedules

In [59]:
nba_sched_url = "https://www.espn.com/nba/schedule/_/date/"
browser.visit(nba_sched_url)
browser.html
html = browser.html
soup = BeautifulSoup(html, 'lxml')
sched_tables = soup.find('table', class_="schedule has-team-logos align-left")
schedules_sites = pd.read_html(sched_tables.prettify())
schedules_sites

[            matchup          Unnamed: 1 time  (PT) nat tv  \
 0     Portland  POR        Indiana  IND       LIVE    TNT   
 1   Sacramento  SAC  Oklahoma City  OKC       LIVE    NaN   
 2  Los Angeles  LAL    Golden State  GS    7:30 PM    TNT   
 
                  tickets  Unnamed: 5  
 0     Buy on Vivid Seats         NaN  
 1     Buy on Vivid Seats         NaN  
 2  Tickets as low as $36         NaN  ]

# Load - Transform

### Load dump to mongo / Clean on the go

In [133]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.nba
# db.nba_players_stg.drop()
# db.nba_news_live.drop()

In [ ]:
base_reference_url = "https://www.landofbasketball.com"
#Traverse to all players
for letter in string.ascii_lowercase[:26]:
    players_url = f"https://www.landofbasketball.com/nba_players_index/letter_{letter}.htm"
    browser.visit(players_url)
    browser.html
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    all_player_table = soup.find_all('div', class_="indice-items")
    for row in all_player_table:
        player_d = {}
        print(row.find('a')['href'], end='')
        player_url = row.find('a')['href'].replace('..', base_reference_url)
        tables = pd.read_html(player_url)
        try_df = tables[1].set_index(0)
        try_df.rename(columns={0: "Key", 1: "Value"}, inplace=True)
        try_df = try_df.to_dict('index')
        player_d['Name'] = tables[3][0][0].split(' Profile',1)[0]
        player_d['Data'] = try_df
        db.nba_players_stg.insert_one(player_d)

### Validate data availability

In [116]:

# result_dict['facts'] = list(db.mars_fact.find({}, {"_id": 0}))
for a in db.nba_players_stg.find({}, {"_id": 0}).limit(2):
    print(a)

{'Name': 'Alaa Abdelnaby', 'Data': {'Position:': {'Value': 'Forward / Center'}, 'Jersey Number:': {'Value': '31 (Trail Blazers), 5 (Bucks), 4 (Celtics), 30 (Kings, 76ers)'}, 'Height:': {'Value': '6-10 / 2.08 mts'}, 'Weight:': {'Value': '240 lbs / 108.9 kg'}, 'Born:': {'Value': 'June 24, 1968 in Cairo, Egypt'}, 'High-School:': {'Value': 'Bloomfield in Bloomfield, New Jersey'}, 'College:': {'Value': 'Duke University'}, 'Drafted:': {'Value': 'by the Portland Trail Blazers with No. 25 overall pick in round 1 of the 1990 NBA Draft'}, 'NBA Seasons:': {'Value': '5 (3 Playoffs)'}, 'NBA Titles:': {'Value': 'none'}, 'All-Star:': {'Value': 'none'}}}
{'Name': 'Zaid Abdul-Aziz', 'Data': {'Position:': {'Value': 'Center / Forward'}, 'Jersey Number:': {'Value': '21 (Royals), 16 (Bucks), 35 (Bucks, Supersonics), 6 (Rockets), 35 (Rockets, Braves), 54 (Celtics), 27 (Rockets)'}, 'Height:': {'Value': '6-9 / 2.06 mts'}, 'Weight:': {'Value': '235 lbs / 106.6 kg'}, 'Born:': {'Value': 'April 07, 1946 in Brookl

## Scrape News

In [134]:
nba_news_url = "https://apnews.com/NBA"
browser.visit(nba_news_url)
browser.html
# time.sleep(10)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_tables = soup.find_all('div', attrs={"data-key":"feed-card-wire-story-with-image"})
# schedules_sites = pd.read_html(sched_tables.prettify())
# schedules_sites


In [ ]:
for row in news_tables:
    news_content = {}
    news_content['headline'] = row.h1.text
    news_content['content'] = row.p.text
    print(f"Loading {row.h1.text}....")
    db.nba_news_live.insert_one(news_content)

### ETL For elections

In [ ]:
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_pd = pd.read_json(geojson_source)

json_file = "etl/gz_2010_us_050_00_500k.json"
customer_location_df = pd.read_json(json_file, encoding="utf-8")

In [76]:
results_file = "etl/countypres_2000-2016.csv"
results_df = pd.read_csv(results_file, encoding="utf-8")
results_df.head()
results_df.dropna(inplace=True)
results_df.drop(columns=['office', 'version'],inplace=True)
results_df.head()

,year,state,state_po,county,FIPS,candidate,party,candidatevotes,totalvotes
0,2000,Alabama,AL,Autauga,1001.0,Al Gore,democrat,4942.0,17208
1,2000,Alabama,AL,Autauga,1001.0,George W. Bush,republican,11993.0,17208
2,2000,Alabama,AL,Autauga,1001.0,Ralph Nader,green,160.0,17208
4,2000,Alabama,AL,Baldwin,1003.0,Al Gore,democrat,13997.0,56480
5,2000,Alabama,AL,Baldwin,1003.0,George W. Bush,republican,40872.0,56480


In [158]:
# results_df.loc[results_df['county'].str.lower() =="Orange".lower(),:]

# Cast float to integer
results_df.FIPS = results_df.FIPS.astype(int)
results_df.candidatevotes = results_df.candidatevotes.astype(int)

# Calculate percent won
results_df["percentwon"] = ((results_df.candidatevotes / results_df.totalvotes) * 100).map("{0:,.2f}".format).astype(float)
# results_df.percentwon = results_df.percentwon.astype(int)
# Drop index
results_df.reset_index(drop=True, inplace=True)

# Test results
# results_df.loc[(results_df['county'].str.lower() =="Orange".lower()) & (results_df['state'] == "California"),:].head(100)
# results_df.head()
results_2016 = results_df.loc[(results_df.year == 2016) & (results_df['state'] == "California"),:]

In [159]:
results_2016.dtypes

year                int64
state              object
state_po           object
county             object
FIPS                int32
candidate          object
party              object
candidatevotes      int32
totalvotes          int64
percentwon        float64
dtype: object

In [94]:
results_df.to_json("etl/processed.json", orient='records')

In [127]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
# geojson_source = "etl/us-county-boundaries.geojson"
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
geojson_source = "etl/ca-us-county-boundaries.csv.csv"
geojson_pd = pd.read_csv(geojson_source, delimiter=';')
geojson_pd.dtypes
geojson_pd = geojson_pd[['GEOID','Geo Shape']]

In [124]:
geojson_pd

,GEOID,NAME,Geo Shape
0,6001,Alameda,"{""type"": ""Polygon"", ""coordinates"": [[[-122.280..."
1,6087,Santa Cruz,"{""type"": ""Polygon"", ""coordinates"": [[[-122.216..."
2,6111,Ventura,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[-1..."
3,6113,Yolo,"{""type"": ""Polygon"", ""coordinates"": [[[-122.164..."
4,6099,Stanislaus,"{""type"": ""Polygon"", ""coordinates"": [[[-120.922..."
5,6047,Merced,"{""type"": ""Polygon"", ""coordinates"": [[[-120.681..."
6,6089,Shasta,"{""type"": ""Polygon"", ""coordinates"": [[[-122.615..."
7,6005,Amador,"{""type"": ""Polygon"", ""coordinates"": [[[-121.027..."
8,6073,San Diego,"{""type"": ""Polygon"", ""coordinates"": [[[-117.437..."
9,6053,Monterey,"{""type"": ""Polygon"", ""coordinates"": [[[-122.026..."


In [160]:
# geojson_pd
results_df
joined_df = results_2016.merge(geojson_pd, how="inner", left_on='FIPS', right_on='GEOID')

In [161]:
# joined_df.loc[joined_df['county'].str.lower() =="Orange".lower()]
# final_df = joined_df.loc[joined_df['county'].str.lower() =="Orange".lower()]
joined_df

,year,state,state_po,county,FIPS,candidate,party,candidatevotes,totalvotes,percentwon,GEOID,Geo Shape
0,2016,California,CA,Alameda,6001,Hillary Clinton,democrat,514842,654266,78.69,6001,"{""type"": ""Polygon"", ""coordinates"": [[[-122.280..."
1,2016,California,CA,Alameda,6001,Donald Trump,republican,95922,654266,14.66,6001,"{""type"": ""Polygon"", ""coordinates"": [[[-122.280..."
2,2016,California,CA,Alpine,6003,Hillary Clinton,democrat,334,602,55.48,6003,"{""type"": ""Polygon"", ""coordinates"": [[[-120.073..."
3,2016,California,CA,Alpine,6003,Donald Trump,republican,217,602,36.05,6003,"{""type"": ""Polygon"", ""coordinates"": [[[-120.073..."
4,2016,California,CA,Amador,6005,Hillary Clinton,democrat,6004,17734,33.86,6005,"{""type"": ""Polygon"", ""coordinates"": [[[-121.027..."
...,...,...,...,...,...,...,...,...,...,...,...,...
111,2016,California,CA,Ventura,6111,Donald Trump,republican,132323,351726,37.62,6111,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[-1..."
112,2016,California,CA,Yolo,6113,Hillary Clinton,democrat,54752,82090,66.70,6113,"{""type"": ""Polygon"", ""coordinates"": [[[-122.164..."
113,2016,California,CA,Yolo,6113,Donald Trump,republican,20739,82090,25.26,6113,"{""type"": ""Polygon"", ""coordinates"": [[[-122.164..."
114,2016,California,CA,Yuba,6115,Hillary Clinton,democrat,7910,22998,34.39,6115,"{""type"": ""Polygon"", ""coordinates"": [[[-121.597..."


In [162]:
def df_to_geojson(df, properties):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{}
#                    'geometry':{}
                  }
        feature['geometry'] = json.loads(row['Geo Shape'])
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [163]:
properties = ['year','state_po','county','candidate','party','candidatevotes','totalvotes','percentwon']
geojson = df_to_geojson(joined_df, properties)

In [165]:
json = json.dumps(geojson)
f = open("etl/processed.json","w")
f.write(json)
f.close()

# output_filename = 'dataset.js'
# with open(output_filename, 'wb') as output_file:
#     output_file.write('var dataset = ')
#     json.dump(geojson, output_file, indent=2) 